In [ ]:
import pandas as pd
from pathlib import Path 
from fastai.vision import *
from fastai.metrics import AUROC
%matplotlib inline

# Data locations
path = Path("data/")

## Collect data

In [ ]:
df = pd.read_csv(path/"train.csv")
df.head()

In [ ]:
tfms = get_transforms(flip_vert=True)

In [ ]:
test = ImageList.from_csv(path, folder="test", csv_name="sample_submission.csv")
train = (ImageList.from_csv(path, folder="train", csv_name="train.csv")
    .split_by_rand_pct(0.01)
    .label_from_df()
    .add_test(test)
    .transform(tfms, size=128)
    .databunch(path=".", bs=64)
    .normalize(imagenet_stats)
        )

In [ ]:
train.show_batch(rows=3, figsize=(7, 7))

## Train learner

In [ ]:
# Using AUROC() works, but auc_roc_curve directly not?
# --> See https://github.com/fastai/fastai/issues/1939
learn = cnn_learner(train, models.resnet50, metrics=AUROC())

In [ ]:
learn.model

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.save("stage-final")

## Visualize

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
losses, idxs = interp.top_losses()
interp.plot_top_losses(9, figsize=(7, 7))

In [ ]:
interp.plot_confusion_matrix(figsize=(5, 5), dpi=60)

## Create test predictions

In [ ]:
preds, targs = learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
submission = pd.read_csv(path/"sample_submission.csv")
submission.head()

In [ ]:
submission["has_cactus"] = preds.clone().detach().numpy().argmax(1)

In [ ]:
submission.head()

In [ ]:
submission.to_csv("submission.csv", index=False)

In [ ]:
!kaggle competitions submit -f submission.csv -m "First submission" aerial-cactus-identification

## Unnecessary

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(1)

In [ ]:
learn.load("stage-1");

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(3, max_lr=slice(1e-6, 1e-3))